In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from src.data_processing.downscaling import resample_image, Downscaler
from src.data_processing.sentinel_preprocessing import load_sentinel2_data

from src.et_green.exporting_utils import get_time_step_pattern, generate_export_task

from utils.date_utils import (
    set_to_first_of_month,
    print_collection_dates,
    create_centered_date_ranges,
    create_forward_date_ranges,
    get_days_in_month,
    merge_same_date_images,
)
from utils.ee_utils import (
    harmonized_ts,
    export_image_to_asset,
    back_to_int,
    back_to_float,
    fill_gaps_with_zeros,
)
from utils.harmonic_regressor import HarmonicRegressor

from typing import List, Callable, Tuple

---

## Constants

In [4]:
YEAR = 2018

PATH_TO_AOI = "projects/thurgau-irrigation/assets/FribourgAndVaud/broye_bounding_box"
PATH_TO_ET_PRODUCT = "projects/thurgau-irrigation/assets/ET_Landsat_monthly_canton_v2"

USE_SR = False if YEAR < 2019 else True
WINDOW_DAYS = 30
BAND_TO_RESAMPLE = "ET"
BANDS_TO_HARMONIZE = ["B3", "B4", "B8", "B11", "B12"]
AGGREGATION_OPTIONS = {
    "agg_type": "mosaic",
    "mosaic_type": "least_cloudy",
}
INDEXES_FOR_HARMONIZATION = ["NDVI", "NDWI", "NDBI"]
INDEPENDENT_BANDS = ["gap_filled_NDVI", "gap_filled_NDBI", "gap_filled_NDWI"]
DEPENDENT_BAND = ["ET"]
NUMBER_OF_IMAGES = 7
TEMPORAL_RESOLUTION = "monthly"

SCALING_FACTOR = 100
DYNAMIC = False
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None

DOWNSCALED_ASSET_PATH = f"projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/monthly_Landsat_10m_{YEAR}"

## Utils

In [5]:
def set_days_property(image: ee.Image) -> ee.Image:
    """
    Sets the 'days' property to the number of days in the month of the image.

    Args:
        ee.Image: Image to set property on

    Returns:
        ee.Image: Image with 'days' property set
    """
    date = ee.Date(image.date())
    days = ee.Number(get_days_in_month(date))
    return image.set("days", days)

## 1. Load ET product

In [6]:
aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

aoi = aoi_geometry.buffer(600)

In [7]:
et_coarse_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    # .filter(ee.Filter.inList("Region", ["Fribourg", "Vaud"]))
    .filterBounds(aoi)
    .filterDate(f"{YEAR}-04-01", f"{YEAR}-10-31")
    .sort("system:time_start")
    .map(lambda img: set_days_property(img))
    .map(
        lambda img: back_to_float(
            img, SCALING_FACTOR, DYNAMIC, SCALING_FACTOR_PROPERTY_NAME
        )
    )
)

et_coarse_collection = (
    merge_same_date_images(et_coarse_collection)
    .map(lambda img: resample_image(img, 100, ["ET"]))
    .map(lambda img: fill_gaps_with_zeros(img))
)

et_coarse_collection = et_coarse_collection

et_coarse_list = et_coarse_collection.toList(et_coarse_collection.size())

print_collection_dates(et_coarse_collection)

Dates of images in the collection:
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01


In [8]:
image_1 = ee.Image(et_coarse_list.get(4))

Map = geemap.Map()

vis_params = {
    "bands": ["ET"],
    "min": 0,
    "max": 2,
    "palette": ["blue", "green", "yellow", "red"],
}

Map.addLayer(image_1, vis_params, "ET")

Map.centerObject(aoi, 10)

Map



Map(center=[46.786627433094885, 6.874230754333321], controls=(WidgetControl(options=['position', 'transparent_…

## 2. Load Sentinel-2 data

In [9]:
s2_collection = load_sentinel2_data((YEAR, YEAR), aoi=aoi, use_SR=USE_SR)

time_intervals = create_forward_date_ranges(et_coarse_list, window_days=WINDOW_DAYS)

s2_harmonized = harmonized_ts(
    masked_collection=s2_collection,
    band_list=BANDS_TO_HARMONIZE,
    time_intervals=time_intervals,
    options=AGGREGATION_OPTIONS,
)

# print_collection_dates(s2_harmonized)
s2_collection.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B10',
 'B11',
 'B12',
 'QA10',
 'QA20',
 'QA60',
 'MSK_CLASSI_OPAQUE',
 'MSK_CLASSI_CIRRUS',
 'MSK_CLASSI_SNOW_ICE',
 'probability',
 'clouds',
 'dark_pixels',
 'cloud_transform',
 'shadows',
 't',
 'constant']

### 2.1 Prepare the Sentinel-2 data for harmonization

In [10]:
def add_temporal_bands(collection: ee.ImageCollection) -> ee.ImageCollection:
    """Add temporal bands to each image in the collection."""
    def _add_bands(image: ee.Image) -> ee.Image:
        date = ee.Date(image.get('system:time_start'))
        years = date.difference(ee.Date('1970-01-01'), 'year')
        
        projection = image.select([0]).projection()
        time_band = ee.Image(years).float().rename('t')
        constant_band = ee.Image.constant(1).rename('constant')
        
        return image.addBands([
            time_band.setDefaultProjection(projection),
            constant_band.setDefaultProjection(projection)
        ])
    
    return collection.map(_add_bands)

s2_harmonized = add_temporal_bands(s2_harmonized)
s2_harmonized.first().bandNames().getInfo()

['B3', 'B4', 'B8', 'B11', 'B12', 't', 'constant']

In [11]:
def compute_vegetation_indexes(image: ee.Image) -> ee.Image:
    """
    Compute vegetation indexes for a given image

    Args:
        image (ee.Image): The image to compute the vegetation indexes for

    Returns:
        ee.Image: The input image with the vegetation indexes

    """
    ndvi = image.normalizedDifference(["B8", "B4"]).rename("NDVI")
    ndwi = image.normalizedDifference(["B3", "B8"]).rename("NDWI")
    ndbi = image.normalizedDifference(["B11", "B8"]).rename("NDBI")
    return image.addBands(ndvi).addBands(ndwi).addBands(ndbi)

s2_harmonized_w_vegetation_indexes = s2_harmonized.map(compute_vegetation_indexes)

In [12]:
# s2_harmonized_w_vegetation_indexes.first().bandNames().getInfo()    

### 2.2 Fit an Harmonic Regressor to the Sentinel-2 data

In [13]:
s2_harmonized_gaps_filled = s2_harmonized_w_vegetation_indexes

for index in INDEXES_FOR_HARMONIZATION:
    regressor = HarmonicRegressor(
        omega=1, max_harmonic_order=2, band_to_harmonize=index
    )

    regressor.fit(s2_harmonized_w_vegetation_indexes)
    fitted_collection = regressor.predict(s2_harmonized_w_vegetation_indexes)

    fitted_collection = fitted_collection.map(
        lambda img: img.select(["fitted"]).rename(f"fitted_{index}")
    )

    s2_harmonized_gaps_filled = s2_harmonized_gaps_filled.map(
        lambda img: img.addBands(
            fitted_collection.filterDate(img.date()).first().select([f"fitted_{index}"])
        )
    )

In [14]:
# s2_harmonized_gaps_filled.first().bandNames().getInfo()

### 2.3 Use the fitted harmonic bands to fill the gaps in the original bands

In [15]:
def fill_gaps(
    img: ee.Image, source_band: str, fill_band: str, output_name: str
) -> ee.Image:
    """Fill gaps in a band with values from another band.

    Args:
        img (ee.Image): Input image containing both bands
        source_band (str): Name of band containing gaps to fill
        fill_band (str): Name of band to use for filling gaps
        output_name (str): Name for the output gap-filled band

    Returns:
        ee.Image: Image with gap-filled band
    """
    scale = img.projection().nominalScale()
    projection = img.projection()
    # Create mask where the source band is invalid (gaps)
    gap_mask = img.select(source_band).mask().Not()

    # Get the source band and fill band
    source = img.select(source_band)
    fill = img.select(fill_band)

    # Fill gaps: use source band where available, fill band where there are gaps
    filled = source.unmask().where(gap_mask, fill).rename(output_name)
    filled = filled.setDefaultProjection(projection).set("scale", scale)

    return filled


def apply_gap_filling(img: ee.Image, indexes: List[str]) -> ee.Image:
    """Apply gap filling to multiple bands.

    Args:
        img (ee.Image): Input image
        indexes (list[str]): List of index names to process (e.g., ['NDVI', 'NDWI', 'NDBI'])

    Returns:
        ee.Image: Original image with added gap-filled bands
    """
    # Start with the original image
    result = img

    # Add each gap-filled band one at a time
    for index in indexes:
        filled_band = fill_gaps(
            img=img,
            source_band=index,
            fill_band=f"fitted_{index}",
            output_name=f"gap_filled_{index}",
        )
        result = result.addBands(filled_band)

    return result


# Apply gap filling to the collection
def process_collection(
    collection: ee.ImageCollection, indexes: List[str]
) -> ee.ImageCollection:
    """Process entire collection by applying gap filling to each image.

    Args:
        collection (ee.ImageCollection): Input collection
        indexes (List[str]): List of index names to process

    Returns:
        ee.ImageCollection: Processed collection with gap-filled bands
    """
    return collection.map(lambda img: apply_gap_filling(img, indexes))


s2_harmonized_gaps_filled = process_collection(s2_harmonized_gaps_filled, INDEXES_FOR_HARMONIZATION)

# s2_harmonized_gaps_filled.first().bandNames().getInfo()

## 3. Downscale the ET product to 10m resolution

In [16]:
def prepare_image_lists(
    s2_indices: ee.ImageCollection,
    independent_vars: ee.ImageCollection,
    dependent_vars: ee.ImageCollection,
) -> Tuple[ee.List, ee.List, ee.List]:
    """
    Convert image collections to lists and verify sizes match.

    Args:
        s2_indices: Sentinel-2 indices collection
        independent_vars: Independent variables collection
        dependent_vars: Dependent variables collection

    Returns:
        Tuple containing the three image lists

    Raises:
        ValueError: If collection sizes don't match
    """
    s2_list = s2_indices.toList(s2_indices.size())
    ind_list = independent_vars.toList(independent_vars.size())
    dep_list = dependent_vars.toList(dependent_vars.size())

    return s2_list, ind_list, dep_list


def perform_downscaling(
    downscaler: Downscaler,
    s2_index: ee.Image,
    ind_vars: ee.Image,
    dep_vars: ee.Image,
    aoi: ee.Geometry,
    scale_coarse: float,
) -> ee.Image:
    """
    Perform downscaling operation on a single timestep.

    Args:
        downscaler: Downscaler object
        s2_index: Sentinel-2 indices image
        ind_vars: Independent variables image
        dep_vars: Dependent variables image
        aoi: Area of interest
        scale_coarse: Coarse resolution scale

    Returns:
        Downscaled image
    """
    return downscaler.downscale(
        coarse_independent_vars=ind_vars,
        coarse_dependent_var=dep_vars,
        fine_independent_vars=s2_index,
        geometry=aoi,
        resolution=scale_coarse,
    )


def process_and_export_downscaled_ET(
    downscaler: Downscaler,
    s2_indices: ee.ImageCollection,
    independent_vars: ee.ImageCollection,
    dependent_vars: ee.ImageCollection,
    aoi: ee.Geometry,
    year: int,
    scale_coarse: float,
    asset_path: str,
    scale_fine: float = 10,
    time_step_type: str = "dekadal",
    number_of_images: int = 36,
) -> List[ee.batch.Task]:
    """
    Process and export downscaled WaPOR ET images to Earth Engine assets.

    Args:
        downscaler: The Downscaler object used to downscale the images
        s2_indices: The Sentinel-2 indices image collection
        independent_vars: The resampled independent variables image collection
        dependent_vars: The dependent variables image collection
        aoi: The area of interest geometry
        year: The year for which the images are processed
        scale_coarse: The scale of the images before downscaling
        asset_path: Base path for asset export
        scale_fine: The scale of the images after downscaling
        time_step_type: Type of time step ("dekadal" or "monthly")

    Returns:
        List of export tasks for the downscaled images

    Raises:
        ValueError: If time_step_type is invalid or collections have different sizes
    """
    # Convert collections to lists and validate
    s2_list, ind_list, dep_list = prepare_image_lists(
        s2_indices, independent_vars, dependent_vars
    )

    tasks = []

    for i in range(number_of_images):
        # Get images for current timestep
        s2_index = ee.Image(s2_list.get(i))
        ind_vars = ee.Image(ind_list.get(i))
        dep_vars = ee.Image(dep_list.get(i))

        # Get date from the S2 image
        date = ee.Date(s2_index.get("system:time_start"))
        time_step_pattern = get_time_step_pattern(date, time_step_type)

        # Perform downscaling
        et_image_downscaled = perform_downscaling(
            downscaler=downscaler,
            s2_index=s2_index,
            ind_vars=ind_vars,
            dep_vars=dep_vars,
            aoi=aoi,
            scale_coarse=scale_coarse,
        )

        # Post-process the downscaled image
        et_image_downscaled = back_to_int(et_image_downscaled, 100)

        # Generate task name and export
        task_name = (
            f"Downscaled_ET_gap_filled_{time_step_type}_{year}_{time_step_pattern}"
        )
        task = generate_export_task(
            et_image_downscaled,
            asset_path,
            task_name,
            year,
            aoi,
            resolution=scale_fine,
        )
        tasks.append(task)

    print(f"Generated {len(tasks)} export tasks for year {year}")
    return tasks

In [17]:
scale = et_coarse_collection.first().projection().nominalScale().getInfo()

# Prepare collections
s2_indices = s2_harmonized_gaps_filled.select(INDEPENDENT_BANDS)
independent_vars = s2_indices.map(
    lambda img: resample_image(img, scale, INDEPENDENT_BANDS)
)
dependent_vars = et_coarse_collection.select(DEPENDENT_BAND)

# Initialize the Downscaler
downscaler = Downscaler(
    independent_bands=INDEPENDENT_BANDS,
    dependent_band=DEPENDENT_BAND[0],
)

# Process and export
# tasks = process_and_export_downscaled_ET(
#     downscaler=downscaler,
#     s2_indices=s2_indices,
#     independent_vars=independent_vars,
#     dependent_vars=dependent_vars,
#     aoi=aoi,
#     year=YEAR,  
#     scale_coarse=scale,
#     asset_path=DOWNSCALED_ASSET_PATH,
#     scale_fine=10, 
#     time_step_type=TEMPORAL_RESOLUTION,  
#     number_of_images=NUMBER_OF_IMAGES,
# )

In [18]:
# independent_vars.first().bandNames().getInfo()

## Sanity checks
### 1. Check the Sentinel-2 harmonization

In [19]:
# s2_harmonized_gaps_filled.size().getInfo()

In [20]:
Map = geemap.Map()

sentinel_vis_params = {
    "bands": ["NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}

fitted_vis_params = {
    "bands": ["fitted_NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}

gap_filled_vis_params = {
    "bands": ["gap_filled_NDVI"],
    "min": 0,
    "max": 1,
    "palette": ["white", "green"],
}


s2_image = ee.Image(s2_harmonized_gaps_filled.filterBounds(aoi).toList(7).get(2))
# s2_image_resampled = ee.Image(independent_vars.toList(36).get(0))

Map.addLayer(s2_image, sentinel_vis_params, "Sentinel 2")
# # Map.addLayer(s2_image, fitted_vis_params, "Sentinel 2 Fitted")
Map.addLayer(s2_image, gap_filled_vis_params, "Sentinel 2 Gap Filled")
# Map.addLayer(s2_image_resampled, gap_filled_vis_params, "Sentinel 2 Resampled")


Map.centerObject(aoi, 12)
Map

Map(center=[46.786627433094885, 6.874230754333321], controls=(WidgetControl(options=['position', 'transparent_…

In [21]:
# print_collection_dates(s2_harmonized_gaps_filled)

### 2. Sanity Check

In [15]:
collection = ee.ImageCollection("projects/thurgau-irrigation/assets/FribourgAndVaud/ET_products/monthly_Landsat_10m_2020").map(lambda img: back_to_float(img, 100))

print_collection_dates(collection)

Dates of images in the collection:
2020-04-16
2020-05-16
2020-06-16
2020-07-16
2020-08-16
2020-09-16
2020-10-16


In [16]:
collection_list = collection.toList(collection.size())


In [ ]:
Map = geemap.Map()

et_vis_params = {
    "bands": ["downscaled"],
    "min": 0,
    "max": 1,
    "palette": ["white", "lightblue", "blue", "green", "yellow", "orange", "red", "darkred"],
}

et_coarse_vis_params = {
    "bands": ["ET"],
    "min": 0,
    "max": 2,
    "palette": ["white", "lightblue", "blue", "green", "yellow", "orange", "red", "darkred"],
}


et_image = ee.Image(collection_list.get(6))
# et_image_coarse = ee.Image(et_coarse_list.get(3))

Map.addLayer(et_image, et_vis_params, "ET")
# Map.addLayer(et_image_coarse, et_coarse_vis_params, "ET Coarse")
# Map.addLayer(aoi, {}, "AOI")

Map.centerObject(aoi, 12)

Map

Map(center=[46.786627433094885, 6.874230754333321], controls=(WidgetControl(options=['position', 'transparent_…

In [25]:
# # Get list of unique dates and merge same-day images
# def merge_daily_images(collection):
#     # Get list of unique dates
#     dates = collection.aggregate_array('system:time_start').distinct()
    
#     def merge_images_for_date(date):
#         # Get images for this date
#         daily_images = collection.filter(ee.Filter.eq('system:time_start', date))
#         # Merge images (mosaic takes the first non-masked pixel)
#         merged = daily_images.mosaic()
#         # Set the date property
#         return merged.set('system:time_start', date)
    
#     # Map merge function over dates
#     merged_collection = ee.ImageCollection(dates.map(lambda date: merge_images_for_date(date)))
#     return merged_collection

# # Apply the merging
# merged_sentinel = merge_daily_images(sentinel_collection)

# # Visualize first merged image
# merged_image = ee.Image(merged_sentinel.first())

# # Print number of images before and after
# print(f"Original collection size: {sentinel_collection.size().getInfo()}")
# print(f"Merged collection size: {merged_sentinel.size().getInfo()}")

In [26]:
# Map = geemap.Map()

# sentinel_vis_params = {
#     "bands": ["B4", "B3", "B2"],
#     "min": 0,
#     "max": 2000,
# }

# sentinel_image = ee.Image(sentinel_collection.first())

# Map.addLayer(sentinel_image, sentinel_vis_params, "Sentinel 2")
# Map.addLayer(merged_image, sentinel_vis_params, "Merged Sentinel 2")

# Map.centerObject(aoi, 12)

# Map